In [ ]:
!pip install openai==0.28

In [ ]:
import openai
import os
import openai
from flask import Flask, request, jsonify

openai.api_key = "your own api_key"
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(response["choices"][0]["message"]["content"])

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?


In [ ]:
# write a function to ask chatGPT question
import openai
def ask_chatgpt(prompt, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    user_prompt = input("Enter your question for ChatGPT: ")
    chatgpt_reply = ask_chatgpt(user_prompt)

    if chatgpt_reply:
        print("ChatGPT's response:\n", chatgpt_reply)


Enter your question for ChatGPT: 1+1
ChatGPT's response:
 2


In [ ]:
# build a chatbot to run query on a database in my Google drive called retail_data.csv

import openai
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Replace with your actual API key
openai.api_key = "your own api key"

# Load the CSV file into a Pandas DataFrame
data_path = '/content/drive/My Drive/retail_data.csv'  # Update with your file path
try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    print(f"Error: File not found at {data_path}")
    exit()


Mounted at /content/drive
Enter your query (or type 'exit' to quit): which country has the highest balance?
Error executing query: name 'df' is not defined
Enter your query (or type 'exit' to quit): exit


In [ ]:
%%capture
!pip install -q langchain
!pip install -q openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your own api key"

In [ ]:
import numpy as np
import pandas as pd
import sqlite3

from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('drive/My Drive/retail_data.csv')
conn = sqlite3.connect('retail_db.sqlite')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS retail (CustomerID int, Country text, balance float, max_spent float, mean_spent float, min_spent float, n_orders int)')
conn.commit()

df.to_sql('retail', conn, if_exists='replace', index = False)

c.execute('''
 SELECT Country FROM retail LIMIT 100
           ''')

for row in c.fetchall():
       print (row)

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [ ]:
import sqlite3
read_sql_query(' SELECT Country FROM retail LIMIT 100;',
               "retail_db.sqlite")

In [ ]:
!pip install langchain-community
!pip install langchain-experimental

In [ ]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain import OpenAI, SQLDatabase
from langchain.llms import OpenAI

input_db = SQLDatabase.from_uri('sqlite:///retail_db.sqlite')
llm_1 = OpenAI(temperature=0.8)

In [ ]:
db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

/usr/local/lib/python3.11/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
db_agent.run("how many rows are there?")



> Entering new SQLDatabaseChain chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM retail
SQLResult: [(3254,)]
Answer:Final answer is 3254.
> Finished chain.


'Final answer is 3254.'

In [ ]:
db_agent.run("how many entries of Country are present?")



> Entering new SQLDatabaseChain chain...
how many entries of Country are present?
SQLQuery:SELECT COUNT(*) FROM retail;
SQLResult: [(3254,)]
Answer:There are 3254 entries of Country present.
> Finished chain.


'There are 3254 entries of Country present.'

In [ ]:
db_agent.run("Which country has the highest balance?")



> Entering new SQLDatabaseChain chain...
Which country has the highest balance?
SQLQuery:SELECT "Country" FROM retail ORDER BY balance DESC LIMIT 1;
SQLResult: [('United Kingdom',)]
Answer:United Kingdom
> Finished chain.


'United Kingdom'

In [ ]:
db_agent.run("What is the total of max spent by customers?")



> Entering new SQLDatabaseChain chain...
What is the total of max spent by customers?
SQLQuery:SELECT SUM(max_spent) AS "Total Max Spent" FROM retail
SQLResult: [(4038644.940000006,)]
Answer:4038644.940000006
> Finished chain.


'4038644.940000006'

In [ ]:
db_agent.run("What is the amount of refund to customers?")



> Entering new SQLDatabaseChain chain...
What is the amount of refund to customers?
SQLQuery:SELECT SUM(total_refunded) FROM retail
SQLResult: [(-193678.79999999987,)]
Answer:The total amount of refunds to customers is $193678.80.
> Finished chain.


'The total amount of refunds to customers is $193678.80.'

In [ ]:
# generate these queries using a chatbot in an user interface?

import openai
import os
from flask import Flask, request, jsonify
import pandas as pd
from google.colab import drive
import numpy as np
import sqlite3
from langchain_experimental.sql import SQLDatabaseChain
from langchain import OpenAI, SQLDatabase

# ... (rest of your imports and setup code)

def chatbot_ui():
    """
    Provides a simple text-based UI for interacting with the chatbot.
    """
    print("Welcome to the Retail Data Chatbot!")
    while True:
        user_input = input("Enter your query (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break

        # Attempt to execute the query directly using Pandas
        try:
            query_result = query_database(user_input)  # Assuming query_database function is defined
            if "Error" not in query_result:  # if no error print the result
                print(query_result)
                continue  # ask again for another query
        except Exception as e:
            print(f"An error occurred during Pandas query: {e}")


        # If direct Pandas query fails, try the SQL database agent
        try:
            print("Attempting SQL query...")  # Indicate SQL query attempt
            query_result = db_agent.run(user_input)
            print(query_result)
            continue

        except Exception as e:
            print(f"An error occurred during SQL query: {e}")

        # If both pandas and SQL fail, fallback to ChatGPT
        print("Falling back to ChatGPT for query conversion...")
        prompt = f"Convert this natural language query about retail data into a SQL query string: {user_input}"
        sql_query = ask_chatgpt(prompt)

        if sql_query:
            try:
                print("Executing SQL query generated by ChatGPT...")
                read_sql_query(sql_query, "retail_db.sqlite")
            except Exception as e:
                print(f"Error executing ChatGPT-generated SQL query: {e}")
        else:
            print("Could not generate a valid SQL query.")

if __name__ == "__main__":
    chatbot_ui()


Welcome to the Retail Data Chatbot!
Enter your query (or type 'exit' to quit): which country has the highest total spent?
Attempting SQL query...


> Entering new SQLDatabaseChain chain...
which country has the highest total spent?
SQLQuery:SELECT "Country", MAX(total_spent) FROM retail
SQLResult: [('United Kingdom', 394689.18)]
Answer:United Kingdom
> Finished chain.
United Kingdom
Enter your query (or type 'exit' to quit): What is the amount of refund to customers?
Attempting SQL query...


> Entering new SQLDatabaseChain chain...
What is the amount of refund to customers?
SQLQuery:SELECT total_refunded FROM retail
SQLResult: [(-77183.6,), (0.0,), (0.0,), (-120.33,), (0.0,), (0.0,), (0.0,), (0.0,), (0.0,), (-22.8,), (0.0,), (0.0,), (0.0,), (0.0,), (-1.95,), (0.0,), (0.0,), (-11.25,), (0.0,), (-17.8,), (0.0,), (0.0,), (0.0,), (0.0,), (0.0,), (0.0,), (-30.98,), (-4.95,), (-12.25,), (-9.95,), (0.0,), (0.0,), (0.0,), (0.0,), (0.0,), (-56.95,), (-39.26,), (0.0,), (0.0,), (-54.75,), (0.0,)